In [1]:
from amadeus import Client, ResponseError
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os

# Load environment variables
load_dotenv()

# Initialize Amadeus Client
amadeus = Client(
    client_id=os.getenv('AMADEUS_KEY'),
    client_secret=os.getenv('AMADEUS_SECRET')
)

# Initialize LLM
llm = ChatOpenAI(temperature=0.7, openai_api_key=os.getenv("OPENAI_API_KEY"))

def fetch_flight_costs(origin, destination, departure_date, adults=1):
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin,
            destinationLocationCode=destination,
            departureDate=departure_date,
            adults=adults
        )
        flights = response.data
        return [
            {
                "origin": origin,
                "destination": flight['itineraries'][0]['segments'][0]['arrival']['iataCode'],
                "price": flight['price']['total'],
                "departure_time": flight['itineraries'][0]['segments'][0]['departure']['at']
            }
            for flight in flights
        ]
    except ResponseError as error:
        print("Error fetching flight costs:", error)
        return []




/tmp/ipykernel_40737/3598770200.py:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, openai_api_key=os.getenv("OPENAI_API_KEY"))


In [2]:
def extract_flight_parameters(user_query):
    """
    Uses the LLM to extract flight search parameters from the user's query.
    """
    prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    Extract flight search parameters from the following user query.
    Return the response as a JSON object with the keys: origin, destination, departure_date, adults.

    - The 'origin' and 'destination' keys must contain valid IATA codes (e.g., 'JFK', 'LAX').
    - The 'departure_date' key must be in 'YYYY-MM-DD' format only. If the query specifies a relative date (e.g., 'next week'), convert it into 'YYYY-MM-DD' format.
    - The 'adults' key must be an integer. If not mentioned, default to 1.

    If any key cannot be determined, set its value to null.

    User Query: {query}
    """
)
    
    formatted_prompt = prompt.format(query=user_query)
    response = llm.predict(formatted_prompt)
    return eval(response)  # Assuming the response is valid JSON-like text

def llm_agent_query_flights(user_query):
    """
    Integrates the LLM for parameter extraction and fetches flight data using Amadeus API.
    """
    # Step 1: Use the LLM to extract parameters
    params = extract_flight_parameters(user_query)
    
    # Validate extracted parameters
    required_keys = ['origin', 'destination', 'departure_date']
    if not all(params.get(key) for key in required_keys):
        return {"error": "Missing required parameters in query. Please provide origin, destination, and departure date."}
    print(params)
    
    # Step 2: Fetch flight data
    flights = fetch_flight_costs(
        origin=params['origin'],
        destination=params['destination'],
        departure_date=params['departure_date'],
        adults=params.get('adults', 1)
    )
    
    return flights

# Example Usage
user_query = "I need a flight from Madrid to Karachi around October 10th, 2025 for 2 people."
print(extract_flight_parameters(user_query))

/tmp/ipykernel_40737/2061364901.py:22: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(formatted_prompt)


{'origin': 'MAD', 'destination': 'KHI', 'departure_date': '2025-10-10', 'adults': 2}


In [3]:
results = llm_agent_query_flights(user_query)

if "error" in results:
    print(results["error"])
else:
    for flight in results:
        print(f"Origin: {flight['origin']}, Destination: {flight['destination']}, "
              f"Price: ${flight['price']}, Departure: {flight['departure_time']}")


{'origin': 'MAD', 'destination': 'KHI', 'departure_date': '2025-10-10', 'adults': 2}
Origin: MAD, Destination: MUC, Price: $699.30, Departure: 2025-10-10T18:20:00
Origin: MAD, Destination: MUC, Price: $783.30, Departure: 2025-10-10T14:55:00
Origin: MAD, Destination: FRA, Price: $795.42, Departure: 2025-10-10T05:55:00
Origin: MAD, Destination: IST, Price: $1060.14, Departure: 2025-10-10T14:30:00
Origin: MAD, Destination: IST, Price: $1060.14, Departure: 2025-10-10T12:00:00
Origin: MAD, Destination: DXB, Price: $1332.96, Departure: 2025-10-10T22:05:00
Origin: MAD, Destination: DXB, Price: $1332.96, Departure: 2025-10-10T15:25:00
Origin: MAD, Destination: DXB, Price: $1332.96, Departure: 2025-10-10T15:25:00
Origin: MAD, Destination: DXB, Price: $1332.96, Departure: 2025-10-10T22:05:00
Origin: MAD, Destination: DXB, Price: $1836.96, Departure: 2025-10-10T22:05:00
Origin: MAD, Destination: FCO, Price: $4066.90, Departure: 2025-10-10T05:50:00
Origin: MAD, Destination: BCN, Price: $4215.14, D